In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from preprocessing_utils import process_sat_data

In [3]:
#import visdom

In [4]:
#vis = visdom.Visdom()

In [5]:
data_train = pd.read_csv('../../train.csv')
data_test = pd.read_csv('../../Track 1//test.csv')

In [6]:
train_data = data_train.copy(deep=True)
test_data = data_test.copy(deep=True)

In [7]:
train_data.head()

,id,epoch,sat_id,x,y,z,Vx,Vy,Vz,x_sim,y_sim,z_sim,Vx_sim,Vy_sim,Vz_sim
0,0,2014-01-01T00:00:00.000,0,-8855.823863,13117.780146,-20728.353233,-0.908303,-3.808436,-2.022083,-8843.131454,13138.221690,-20741.615306,-0.907527,-3.804930,-2.024133
1,1,2014-01-01T00:46:43.000,0,-10567.672384,1619.746066,-24451.813271,-0.302590,-4.272617,-0.612796,-10555.500066,1649.289367,-24473.089556,-0.303704,-4.269816,-0.616468
2,2,2014-01-01T01:33:26.001,0,-10578.684043,-10180.467460,-24238.280949,0.277435,-4.047522,0.723155,-10571.858472,-10145.939908,-24271.169776,0.274880,-4.046788,0.718768
3,3,2014-01-01T02:20:09.001,0,-9148.251857,-20651.437460,-20720.381279,0.715600,-3.373762,1.722115,-9149.620794,-20618.200201,-20765.019094,0.712437,-3.375202,1.718306
4,4,2014-01-01T03:06:52.002,0,-6719.092336,-28929.061629,-14938.907967,0.992507,-2.519732,2.344703,-6729.358857,-28902.271436,-14992.399986,0.989382,-2.522618,2.342237


In [8]:
process_sat_data(train_data, test_data, scale=10000)

In [9]:
train_data.head()

,id,epoch,sat_id,x,y,z,Vx,Vy,Vz,x_sim,y_sim,z_sim,Vx_sim,Vy_sim,Vz_sim,cluster
0,0,0.0000,0,-0.885582,1.311778,-2.072835,-0.908303,-3.808436,-2.022083,-0.884313,1.313822,-2.074162,-0.907527,-3.804930,-2.024133,1.0
1,1,0.2803,0,-1.056767,0.161975,-2.445181,-0.302590,-4.272617,-0.612796,-1.055550,0.164929,-2.447309,-0.303704,-4.269816,-0.616468,2.0
2,2,0.5606,0,-1.057868,-1.018047,-2.423828,0.277435,-4.047522,0.723155,-1.057186,-1.014594,-2.427117,0.274880,-4.046788,0.718768,3.0
3,3,0.8409,0,-0.914825,-2.065144,-2.072038,0.715600,-3.373762,1.722115,-0.914962,-2.061820,-2.076502,0.712437,-3.375202,1.718306,4.0
4,4,1.1212,0,-0.671909,-2.892906,-1.493891,0.992507,-2.519732,2.344703,-0.672936,-2.890227,-1.499240,0.989382,-2.522618,2.342237,5.0


In [10]:
def plot_axis(sat_id, axis, ellipse_index, span=1):
    data = train_data[train_data['sat_id'] == sat_id]
    label = data['cluster']
    time = data['epoch']
    data = data.loc[:, ['x_sim', 'y_sim', 'z_sim']].values
    vis.line(data[ellipse_index * 24: (ellipse_index + span) * 24, axis], time[ellipse_index * 24: (ellipse_index + span) * 24], opts={'markersize': 4}, win='Axis'+str(axis))

In [11]:
# sat_id = 1
# plot_axis(sat_id, 0, 0, 11)
# plot_axis(sat_id, 1, 0, 11)
# plot_axis(sat_id, 2, 0, 11)

In [12]:
import torch
import torch.nn as nn

In [13]:
from torch.utils.data import DataLoader
import torch.optim as optim

In [24]:
class SatNet(nn.Module):
    def __init__(self, name, omega, mean, diff):
        super(SatNet, self).__init__()
        self.name = name
        self.w = nn.Parameter(omega)
        self.off = nn.Parameter(mean)
        self.a1 = nn.Parameter(diff)
        self.a2 = nn.Parameter(diff)
        self.p1 = nn.Parameter(torch.randn(1))
        self.p2 = nn.Parameter(torch.randn(1))
    def forward(self, t):
        s1 = self.a1 * torch.cos(self.p1 + self.w * t)
        #s2 = self.a2 * torch.cos(self.p2 + self.w * t * 2)
        s = self.off + s1 #+ s2
        v1 = - self.a1 * self.w * torch.sin(self.w * t + self.p1)
        #v2 = - self.a2 * self.w * torch.sin(self.w * t + self.p2) * 2
        v = v1 #+ v2
        return [s, v]

In [25]:
def smape(satellite_predicted_values, satellite_true_values):  
    return np.mean(np.abs((satellite_predicted_values - satellite_true_values) 
        / (np.abs(satellite_predicted_values) + np.abs(satellite_true_values))))

In [26]:
def get_initial_values(ellipse_data):
    omega = 2 * np.pi / (torch.max(ellipse_data[:, 6]) - torch.min(ellipse_data[:, 6])) # wT = 2(pi)
    mean = torch.mean(ellipse_data[:,[0,1,2]], dim=0)
    amplitude_x = 0.5 * (torch.max(ellipse_data[:,0]) - torch.min(ellipse_data[:,0]))
    amplitude_y = 0.5 * (torch.max(ellipse_data[:,1]) - torch.min(ellipse_data[:,1]))
    amplitude_z = 0.5 * (torch.max(ellipse_data[:,2]) - torch.min(ellipse_data[:,2]))
    return [omega, mean, amplitude_x, amplitude_y, amplitude_z]

In [27]:
def print_status(itr, ellipse_data, loss_x, loss_y, loss_z,smape_x, smape_y, smape_z):
    print('Iteration: ', itr)
    print('Loss: {} (X) | {} (Y) | {} (Z)'.format(loss_x.item(), loss_y.item(), loss_z.item()))
    print('SMAPE: {} (X) | {} (Y) | {} (Z)'.format(smape_x, smape_y, smape_z))

In [28]:
df_cols = ['sat_id', 'ellipse_id', 'smape_x', 'smape_y', 'smape_z',
           'a1_x', 'a2_x', 'p1_x', 'p2_x', 'w_x', 'off_x',
           'a1_y', 'a2_y', 'p1_y', 'p2_y', 'w_y', 'off_y',
           'a1_z', 'a2_z', 'p1_z', 'p2_z', 'w_z', 'off_z']

In [32]:
def save_weights(sat_nets, sat_id, ellipse_id, smape_x, smape_y, smape_z, weights):
    row = [sat_id, ellipse_id, smape_x, smape_y, smape_z]
    for axis in ['x', 'y', 'z']:
        model = sat_nets[sat_id][ellipse_id][axis]
        params = [model.a1, model.a2, model.p1, model.p2, model.w, model.off]
        params = [item.item() for item in params]
        row += params
    weights = np.concatenate([weights.values, [row]], axis=0)
    weights = pd.DataFrame(weights, columns=df_cols)
    weights.to_excel('../../weights.xlsx', index=False)
    return weights

In [43]:
def torch_smape_loss(satellite_predicted_values,satellite_true_values):
    return torch.mean(torch.abs((satellite_predicted_values - satellite_true_values) 
        / (torch.abs(satellite_predicted_values) + torch.abs(satellite_true_values))))

In [51]:
def train(train_data):
    sat_nets = {}
    weights = pd.DataFrame([[-1] * len(df_cols)], columns=df_cols)
    weights.to_excel('../../weights.xlsx', index=False)
    for sat_id in range(6,7):
        
        print('------- Satellite ID:', sat_id, '------- ')
        cols = ['x_sim', 'y_sim', 'z_sim', 'Vx_sim', 'Vy_sim', 'Vz_sim', 'epoch']
        sat_data = train_data[train_data['sat_id'] == sat_id].loc[:, cols].values
        print('\n## Number of Ellipses: ', int(sat_data.shape[0]/24)+1, '\n')
        sat_nets[sat_id] = {}
        data = DataLoader(sat_data, batch_size=24) # 24 points per ellipse
        data_iters = iter(data)
        
        for ellipse_id, ellipse_data in enumerate(data_iters):
            
            print('\n**** Ellipse ID:', ellipse_id, '****\n')
            ellipse_data = ellipse_data.float()
            omega, mean, amplitude_x, amplitude_y, amplitude_z = get_initial_values(ellipse_data)
            sat_nets[sat_id][ellipse_id] = {
                'x': SatNet('x', omega, mean[0], amplitude_x),
                'y': SatNet('y', omega, mean[1], amplitude_y),
                'z': SatNet('z', omega, mean[2], amplitude_z)
            }
            
            optim_x = optim.Adam(sat_nets[sat_id][ellipse_id]['x'].parameters(), 0.005, (0.9, 0.999))
            optim_y = optim.Adam(sat_nets[sat_id][ellipse_id]['y'].parameters(), 0.005, (0.9, 0.999))
            optim_z = optim.Adam(sat_nets[sat_id][ellipse_id]['z'].parameters(), 0.005, (0.9, 0.999))
            
            for itr in range(9000):
                optim_x.zero_grad()
                optim_y.zero_grad()
                optim_z.zero_grad()
                
                time = ellipse_data[:,6].float()
                x_pred, vx_pred = sat_nets[sat_id][ellipse_id]['x'](time)
                y_pred, vy_pred = sat_nets[sat_id][ellipse_id]['y'](time)
                z_pred, vz_pred = sat_nets[sat_id][ellipse_id]['z'](time)
                
                #loss_sx = torch.mean((x_pred - ellipse_data[:,0]) ** 2)
                loss_sx = torch_smape_loss(x_pred,ellipse_data[:,0])
                loss_sy = torch_smape_loss(y_pred,ellipse_data[:,1])
                loss_sz = torch_smape_loss(z_pred,ellipse_data[:,2]) 
                loss_vx = torch_smape_loss(vx_pred,ellipse_data[:,3])
                loss_vy = torch_smape_loss(vy_pred,ellipse_data[:,4])
                loss_vz = torch_smape_loss(vz_pred,ellipse_data[:,5])
                loss_x = loss_sx + loss_vx
                loss_y = loss_sy + loss_vy
                loss_z = loss_sz + loss_vz
                
                loss_x.backward()
                loss_y.backward()
                loss_z.backward()
                
                optim_x.step()
                optim_y.step()
                optim_z.step()
                
                if(itr % 1000 == 0):
                    smape_x = smape(x_pred.detach().numpy(), ellipse_data[:,0].numpy())
                    smape_y = smape(y_pred.detach().numpy(), ellipse_data[:,1].numpy())
                    smape_z = smape(z_pred.detach().numpy(), ellipse_data[:,2].numpy())
                    print_status(itr, ellipse_data, loss_x, loss_y, loss_z, smape_x, smape_y, smape_z)
                    
            weights = save_weights(sat_nets, sat_id, ellipse_id, smape_x, smape_y, smape_z, weights)

In [ ]:
sat_nets = train(train_data)

------- Satellite ID: 6 ------- 

## Number of Ellipses:  18 


**** Ellipse ID: 0 ****

Iteration:  0
Loss: 1.6031842231750488 (X) | 1.7112078666687012 (Y) | 0.9856246709823608 (Z)
SMAPE: 0.796480655670166 (X) | 0.7816153168678284 (Y) | 0.4952806234359741 (Z)
Iteration:  1000
Loss: 0.35832294821739197 (X) | 0.2835368514060974 (Y) | 0.18448415398597717 (Z)
SMAPE: 0.10391455888748169 (X) | 0.14841847121715546 (Y) | 0.05764235928654671 (Z)
Iteration:  2000
Loss: 0.3566218316555023 (X) | 0.32719677686691284 (Y) | 0.22472769021987915 (Z)
SMAPE: 0.11937955021858215 (X) | 0.17335838079452515 (Y) | 0.09882930666208267 (Z)
Iteration:  3000
Loss: 0.3450889587402344 (X) | 0.2758845090866089 (Y) | 0.1686023473739624 (Z)
SMAPE: 0.11913704872131348 (X) | 0.13957522809505463 (Y) | 0.05453008413314819 (Z)
Iteration:  4000
Loss: 0.33155137300491333 (X) | 0.27555182576179504 (Y) | 0.16488143801689148 (Z)
SMAPE: 0.10476452112197876 (X) | 0.14237026870250702 (Y) | 0.04837143421173096 (Z)
Iteration:  5000